In [1]:
for label in ["minus", "plus"]:
    with open(f"../data/dms_signal/GSM1297510_Yeast_10c_{label}.wig") as f:
        lines = f.readlines()
    lines = [l.strip().replace("\t", "") for l in lines[1:]]
    table = ["chr\tpos\tsignal\n"]
    for l in lines:
        if l.startswith("variableStep"):
            chr = l.split(" ")[1].split("=")[1]
        else:
            pos, signal = l.split(" ")
            table.append(f"{chr}\t{pos}\t{signal}\n")
    print(table[:5])
    with open(f"../data/dms_signal/{label}.tsv", "w") as f:
        f.writelines(table)

['chr\tpos\tsignal\n', 'chrI\t2953\t1.0\n', 'chrI\t3637\t1.0\n', 'chrI\t3639\t1.0\n', 'chrI\t3793\t1.0\n']
['chr\tpos\tsignal\n', 'chrI\t794\t1.0\n', 'chrI\t5881\t1.0\n', 'chrI\t6753\t1.0\n', 'chrI\t6984\t1.0\n']


In [2]:
import pandas as pd

In [3]:
df1 = pd.read_csv("../data/dms_signal/plus.tsv", sep="\t")
df2 = pd.read_csv("../data/dms_signal/minus.tsv", sep="\t")

In [4]:
df = pd.concat([df1, df2])

In [5]:
df1.shape, df2.shape, df.shape

((1236653, 3), (1217741, 3), (2454394, 3))

In [6]:
df = df.sort_values(["chr", "pos"])

In [7]:
df.to_csv("../data/dms_signal/combined.tsv", sep="\t", index=False)